In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Merging of price and reports

TO DO:

- why do I get INF values to some values?? Becuase some accounting variables is 0
- I belive there are many missing FCF numbers, thats why the low efficacy.
- Fix the warnings
- add gross profit margin (gross profit/sales), what about the very large values??



In [2]:
price_df = pd.read_csv('data/SWE_PRICE_20YEAR.csv')
price_df['d'] = pd.to_datetime(price_df['d'])
price_df = price_df.set_index(['d', 'insID'])

In [3]:
reports_df = pd.read_csv('data/SWE_REPORTS.csv')
reports_df['d'] = pd.to_datetime(reports_df['d'])
reports_df = reports_df.set_index(['d', 'insID'])

In [4]:
reports_df.columns

Index(['broken_Fiscal_Year', 'cash_And_Equivalents', 'cash_Flow_For_The_Year',
       'cash_Flow_From_Financing_Activities',
       'cash_Flow_From_Investing_Activities',
       'cash_Flow_From_Operating_Activities', 'currency', 'currency_Ratio',
       'current_Assets', 'current_Liabilities', 'dividend',
       'earnings_Per_Share', 'financial_Assets', 'free_Cash_Flow',
       'gross_Income', 'intangible_Assets', 'net_Debt', 'net_Sales',
       'non_Current_Assets', 'non_Current_Liabilities', 'number_Of_Shares',
       'operating_Income', 'period', 'profit_Before_Tax',
       'profit_To_Equity_Holders', 'report_Date', 'report_End_Date',
       'report_Start_Date', 'revenues', 'stock_Price_Average',
       'stock_Price_High', 'stock_Price_Low', 'tangible_Assets',
       'total_Assets', 'total_Equity', 'total_Liabilities_And_Equity', 'year'],
      dtype='object')

In [5]:
#left merge reports on price data
complete_df =  price_df.join(reports_df, on=['d','insID'])


## FORWARD FILL
complete_daily = complete_df.groupby(level='insID').ffill()
complete_df =complete_daily.groupby([complete_daily.index.get_level_values(1), pd.Grouper(freq='m', level=0)]).last()
complete_df = complete_df.swaplevel()


In [6]:
PFCF_df = complete_df[['cash_Flow_For_The_Year','c', 'number_Of_Shares']]
#PFCF_df = PFCF_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
PFCF_df['fcf_per_share'] = PFCF_df['cash_Flow_For_The_Year']/PFCF_df['number_Of_Shares']
PFCF_df['fcf_r12'] = PFCF_df['fcf_per_share'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### Extract convexity

## TEST OF RESIDUAL MOMENTUM

In [7]:
res_tmp = pd.DataFrame(price_df['c'])
res_tmp = res_tmp.unstack('insID')

index_df = pd.read_csv('data/OMX_GI.csv')
index_df['d'] = pd.to_datetime(index_df['d'])
index_df = index_df.set_index(['d'])
index_df = pd.DataFrame(index_df['c'])
index_df = index_df.resample('m').last()
index_df.columns = ['INDEX']

res_tmp = res_tmp.join(index_df, on=['d'],how='left')

res_log = np.log(res_tmp) - np.log(res_tmp.shift(1))
res_log = res_log.loc[res_log['INDEX'].first_valid_index():,]




/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:9138: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  sort=sort,


In [8]:
res_log

,"(c, 2)","(c, 3)","(c, 6)","(c, 7)","(c, 8)","(c, 10)","(c, 13)","(c, 14)","(c, 17)","(c, 18)",...,"(c, 2329)","(c, 2330)","(c, 2331)","(c, 2332)","(c, 2335)","(c, 2336)","(c, 2346)","(c, 2347)","(c, 2359)",INDEX
d,,,,,,,,,,,,,,,,,,,,,
2002-02-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-05,NaN,-0.058841,-0.009478,NaN,0.002319,NaN,-0.020203,NaN,0.010399,0.011788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-06,NaN,0.018020,0.004751,NaN,-0.047638,NaN,0.034387,NaN,0.003443,-0.011788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-07,NaN,0.029327,-0.004751,0.040005,0.000000,NaN,0.081126,NaN,-0.042111,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-08,NaN,-0.011627,-0.058840,-0.165985,0.024117,NaN,-0.028988,NaN,-0.043963,0.001974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-27,-0.004321,-0.011870,0.026983,-0.004409,-0.024391,0.001274,-0.013793,0.043952,-0.013765,0.042427,...,0.005990,-0.010101,-0.032925,-0.003503,0.010969,0.000000,-0.028931,-0.078781,NaN,NaN
2022-01-28,-0.001444,-0.035846,-0.033844,0.021851,0.024391,-0.014107,0.067139,-0.046044,0.004068,-0.001440,...,-0.033496,-0.023108,-0.004193,-0.012357,-0.018349,0.011050,-0.016186,0.006536,-0.042925,NaN
2022-01-31,0.005190,0.011674,-0.029697,0.036779,0.023811,0.005795,-0.050610,-0.003146,0.026839,-0.026644,...,0.015222,0.002594,-0.004211,0.007080,0.000000,0.008208,0.040129,0.009725,-0.076540,NaN


In [9]:
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm


res_tmp = pd.DataFrame(price_df['c'])
res_tmp =res_tmp.groupby([res_tmp.index.get_level_values(1), pd.Grouper(freq='m', level=0)]).last()
res_tmp = res_tmp.unstack('insID')
res_tmp.columns = res_tmp.columns.get_level_values(1)

index_df = pd.read_csv('data/OMX_GI.csv')
index_df['d'] = pd.to_datetime(index_df['d'])
index_df = index_df.set_index(['d'])
index_df = pd.DataFrame(index_df['c'])
index_df = index_df.resample('m').last()
index_df.columns = ['INDEX']

res_tmp = res_tmp.join(index_df, on=['d'],how='left')

res_log = np.log(res_tmp) - np.log(res_tmp.shift(1))
res_log = res_log.loc[res_log['INDEX'].first_valid_index():,]


norm_res_df = pd.DataFrame()
ido_mom_df = pd.DataFrame()


for col in res_log.columns:
    try:
        X = sm.add_constant(res_log['INDEX'])
        Y = res_log[col]
        mod = RollingOLS(endog=Y, exog=X, window=36).fit(cov_type='HCO')
        Y_hat = mod.params['const'] + mod.params['INDEX']*res_log['INDEX']
        ind_residual = Y - Y_hat
        
        #HANDLE OUTLIERS??
        norm_res_df.loc[:,col] = ind_residual.rolling(12).mean()/ind_residual.rolling(12).std()
        ido_mom_df.loc[:,col] = ind_residual.rolling(12).mean()
        
    except:
        print("Error: ", col)
        norm_res_df.loc[:,col] = np.nan
        ido_mom_df.loc[:,col] = np.nan
        

temp_price_df  = pd.DataFrame(price_df['c'])


norm_res_df = norm_res_df.drop(["INDEX"], axis=1)       
norm_res_df['res_mom12'] = 'res_mom12'
norm_res_df = norm_res_df.set_index('res_mom12', append=True).unstack('res_mom12')
norm_res_df.columns = norm_res_df.columns.set_names(['insID', 'res_mom12'])
norm_res_df = norm_res_df.stack('insID')
norm_res_df.columns = norm_res_df.columns.set_names(None)
norm_res_df = temp_price_df.merge(norm_res_df, on = ['insID', 'd'], how="outer").swaplevel().sort_index()
norm_res_df = norm_res_df.groupby('insID').ffill()
norm_res_df = norm_res_df.unstack(level ='d').fillna(norm_res_df.unstack(level ='d').median()).stack(level ='d')
norm_res_df = pd.DataFrame(norm_res_df[['res_mom12']])


ido_mom_df = ido_mom_df.drop(["INDEX"], axis=1)       
ido_mom_df['ido_mom12'] = 'ido_mom12'
ido_mom_df = ido_mom_df.set_index('ido_mom12', append=True).unstack('ido_mom12')
ido_mom_df.columns = ido_mom_df.columns.set_names(['insID', 'ido_mom12'])
ido_mom_df = ido_mom_df.stack('insID')
ido_mom_df.columns = ido_mom_df.columns.set_names(None)
ido_mom_df = temp_price_df.merge(ido_mom_df, on = ['insID', 'd'], how="outer").swaplevel().sort_index()
ido_mom_df = ido_mom_df.groupby('insID').ffill()
ido_mom_df = ido_mom_df.unstack(level ='d').fillna(ido_mom_df.unstack(level ='d').median()).stack(level ='d')
ido_mom_df = pd.DataFrame(ido_mom_df[['ido_mom12']])


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: invalid value encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


In [10]:
### 12 Month Alpha and Beta
res_tmp = pd.DataFrame(price_df['c'])
res_tmp =res_tmp.groupby([res_tmp.index.get_level_values(1), pd.Grouper(freq='m', level=0)]).last()
res_tmp = res_tmp.unstack('insID')
res_tmp.columns = res_tmp.columns.get_level_values(1)

index_df = pd.read_csv('data/OMX_GI.csv')
index_df['d'] = pd.to_datetime(index_df['d'])
index_df = index_df.set_index(['d'])
index_df = pd.DataFrame(index_df['c'])
index_df = index_df.resample('m').last()
index_df.columns = ['INDEX']

res_tmp = res_tmp.join(index_df, on=['d'],how='left')

res_log = np.log(res_tmp) - np.log(res_tmp.shift(1))
res_log = res_log.loc[res_log['INDEX'].first_valid_index():,]


alpha_df = pd.DataFrame()
beta_df = pd.DataFrame()


for col in res_log.columns:
    try:
        X = sm.add_constant(res_log['INDEX'])
        Y = res_log[col]
        mod = RollingOLS(endog=Y, exog=X, window=36).fit(cov_type='HCO')
        
        alpha = mod.params['const']
        beta = mod.params['INDEX']
        
        alpha_df.loc[:,col] = alpha
        beta_df.loc[:,col] = beta
  
        
    except:
        #print("Error: ", col)
        alpha_df.loc[:,col] = np.nan
        beta_df.loc[:,col] = np.nan

        
#### FORMAT DATA FRAMES
temp_price_df  = pd.DataFrame(price_df['c'])

alpha_df = alpha_df.drop(["INDEX"], axis=1)       
alpha_df['alpha'] = 'alpha'
alpha_df = alpha_df.set_index('alpha', append=True).unstack('alpha')
alpha_df.columns = alpha_df.columns.set_names(['insID', 'alpha'])
alpha_df = alpha_df.stack('insID')
alpha_df.columns = alpha_df.columns.set_names(None)
alpha_df = temp_price_df.merge(alpha_df, on = ['insID', 'd'], how="outer").swaplevel().sort_index()
alpha_df = alpha_df.groupby('insID').ffill()
alpha_df = alpha_df.unstack(level ='d').fillna(alpha_df.unstack(level ='d').median()).stack(level ='d')
alpha_df['volatility'] = alpha_df['c'].pct_change().groupby(level='insID').apply(lambda x: x.rolling(252).std()*252**0.5)
alpha_df = alpha_df.groupby(level='insID').apply(
    lambda x: x.assign(std_alpha = x.alpha/x.volatility))

alpha_df = pd.DataFrame(alpha_df[['alpha','std_alpha']])


beta_df = beta_df.drop(["INDEX"], axis=1)       
beta_df['beta'] = 'beta'
beta_df = beta_df.set_index('beta', append=True).unstack('beta')
beta_df.columns = beta_df.columns.set_names(['insID', 'beta'])
beta_df = beta_df.stack('insID')
beta_df.columns = beta_df.columns.set_names(None)
beta_df = temp_price_df.merge(beta_df, on = ['insID', 'd'], how="outer").swaplevel().sort_index()
beta_df = beta_df.groupby('insID').ffill()
beta_df = beta_df.unstack(level ='d').fillna(beta_df.unstack(level ='d').median()).stack(level ='d')
beta_df = pd.DataFrame(beta_df[['beta']])

## All Around Metrics

In [11]:
#Market Cap
mcap_df = complete_df[['c','number_Of_Shares']]
#mcap_df = mcap_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')

### Backwardsfill missing numbers of shares
mcap_df = mcap_df.unstack(level ='insID').fillna(method ='bfill').stack(level ='insID')

####

mcap_df['MCAP'] = mcap_df['c'] * mcap_df['number_Of_Shares']
mcap_df = mcap_df.sort_index() 

In [12]:
volume_df = complete_df[['c', 'v', 'number_Of_Shares']]
#volume_df = volume_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
volume_df['MCAP'] = volume_df['c'] * volume_df['number_Of_Shares']
volume_df['VOLUME'] = volume_df['c'] * volume_df['v']

volume_df['MA3_VOL'] = volume_df['VOLUME'].groupby(level='insID').apply(lambda x: x.rolling(3).mean())
volume_df = volume_df.unstack(level ='d').fillna(volume_df.unstack(level ='d').median()).stack(level ='d')
volume_df = volume_df[['MA3_VOL']]
volume_df = volume_df.sort_index()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [13]:
#Cumulative dividends
div_adj_df = complete_df[['dividend', 'c']]
div_adj_df.loc[div_adj_df['dividend'].isna(),['dividend']] = 0
div_adj_df['rol_div'] = div_adj_df['dividend'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
div_adj_df.loc[div_adj_df['rol_div'].isna(),['rol_div']] = 0

div_adj_df = div_adj_df.groupby(level='insID').apply(
    lambda x: x.assign(day_div = x.rol_div/12))

div_adj_df['cum_day_div'] = div_adj_df['day_div'].groupby(level='insID').cumsum()
div_adj_df = div_adj_df.groupby(level='insID').apply(
    lambda x: x.assign(adj_c = x.c+ x.cum_day_div))

div_adj_df = div_adj_df['adj_c']

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
#rename lists and sectors

In [15]:
list_sector_df = complete_df[['Name','sectorId', 'marketId']]


list_sector_df['sector'] = np.nan
list_sector_df.loc[: , 'market']= list_sector_df['marketId'].replace({1: 'Large_Cap', 2: 'Mid_Cap',
                                                               3: 'Small_Cap', 4: 'First_North'})

list_sector_df.loc[: , 'sector']= list_sector_df['sectorId'].replace({1: 'finance_RE', 2: 'consumer_goods',
                                                               3: 'energy', 4: 'healthcare',
                                                                     5: 'industry', 6:'info_tech',
                                                                     7:'material', 8:'consumer_discretionary',
                                                                     9:'telecom', 10:'infrastructure'})



list_sector_df = list_sector_df[['Name','sector', 'market']]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

## Calculate EAR and SUE seperatly

In [23]:
EAR_date_df = pd.DataFrame(reports_df['report_Date'])
EAR_date_df['EA_ind'] = 1
EAR_date_df = price_df.join(EAR_date_df, on=['d','insID'])[['c','EA_ind' ]]
EAR_date_df['EA_plus'] = EAR_date_df['EA_ind'].groupby(level='insID').shift(1)
EAR_date_df = EAR_date_df[~EAR_date_df.index.duplicated(keep='last')]

#Calculate 3 day return, only select the days where EAR_date_df['EA_plus'] == 1
EAR_date_df['3day_ret'] = EAR_date_df['c'].groupby(level='insID').pct_change(3)
EAR_date_df['EA_ret'] = EAR_date_df['3day_ret'] * EAR_date_df['EA_plus']
EAR_date_df = EAR_date_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
EAR_date_df = EAR_date_df.unstack(level ='d').fillna(EAR_date_df.unstack(level ='d').median()).stack(level ='d')

EAR_date_df['volatility_6m'] = EAR_date_df['c'].pct_change().groupby(level='insID').apply(lambda x: x.rolling(6).std()*12**0.5)
EAR_date_df.loc[EAR_date_df['volatility_6m'] <= 0.05 ,'volatility_6m'] = 0.05

EAR_date_df['EAR_std'] = EAR_date_df['EA_ret']/(EAR_date_df['volatility_6m'] )


EAR_date_df = EAR_date_df[['EA_ret', 'EAR_std']]
EAR_date_df =EAR_date_df.groupby([EAR_date_df.index.get_level_values(0), pd.Grouper(freq='m', level=1)]).last()
EAR_date_df = test.swaplevel()
EAR_date_df = EAR_date_df.sort_index()



In [24]:
#RSURP YEAR
rev_year = complete_df[['c','revenues']]
#rev_year = rev_year.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')

rev_year.loc[rev_year['revenues'] == 0,'revenues'] =-0.001 #imputing to remove inf
rev_year['rev_vol'] = rev_year['revenues'].groupby(level='insID').apply(lambda x: x.rolling(12*2).std())
rev_year['rev_growth'] = rev_year['revenues'].groupby(level='insID').pct_change(12)
rev_year['RSURP_Y'] = rev_year['rev_growth']/rev_year['rev_vol']
rev_year = rev_year.unstack(level ='d').fillna(rev_year.unstack(level ='d').median()).stack(level ='d')
rev_year = rev_year[['RSURP_Y','rev_growth']]

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [25]:
#SUE YEAR
eps_year = complete_df[['c','earnings_Per_Share']]

eps_year.loc[eps_year['earnings_Per_Share'] == 0,'earnings_Per_Share'] =-0.001 #imputing to remove inf

eps_year['EPS_vol'] = eps_year['earnings_Per_Share'].groupby(level='insID').apply(lambda x: x.rolling(12*2).std())
eps_year['EPS_growth'] = eps_year['earnings_Per_Share'].groupby(level='insID').pct_change(12)

eps_year.loc[eps_year['EPS_vol'] <= 0.05 ,'EPS_vol'] = 0.05
#eps_year.loc[eps_year['EPS_growth'] <= 0.05 ,'EPS_growth'] = 0.05

eps_year['SUE_Y'] = eps_year['EPS_growth']/eps_year['EPS_vol']
eps_year['EPS_acceleration'] = eps_year['EPS_growth'].diff(252)
eps_year = eps_year.unstack(level ='d').fillna(eps_year.unstack(level ='d').median()).stack(level ='d')
eps_year = eps_year[['SUE_Y','EPS_growth', 'EPS_acceleration']]



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
complete_df =complete_daily.groupby([complete_daily.index.get_level_values(1), pd.Grouper(freq='m', level=0)]).last()
complete_df = complete_df.swaplevel()
complete_daily

In [27]:
252/2

126.0

In [29]:
#MOMENTUM & vol MEASURES
mom_df = complete_daily[['c']]
#mom_df = mom_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
mom_df['mom12'] = mom_df.c.groupby(level='insID').pct_change(252)
mom_df['mom6'] = mom_df.c.groupby(level='insID').pct_change(126)
mom_df['mom3'] = mom_df.c.groupby(level='insID').pct_change(63)
mom_df['mom1'] = mom_df.c.groupby(level='insID').pct_change(21)
mom_df['1_m_ret'] = mom_df.c.groupby(level='insID').pct_change()
mom_df['volatility'] = mom_df['1_m_ret'].groupby(level='insID').apply(lambda x: x.rolling(12).std()*12**0.5)



#### RELATIVE TO ROLLING HIGH
mom_df['high_4y'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(252*4).max())
mom_df['high_3y'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(252*3).max())
mom_df['high_2y'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(252*2).max())
mom_df['high_12m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(262).max())
mom_df['high_6m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(126).max())
mom_df['high_3m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(63).max())
mom_df['high_1m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(21).max())

#### RELATIVE TO ROLLING low
mom_df['low_2y'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(252*2).min())
mom_df['low_12m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(252).min())
mom_df['low_6m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(126).min())
mom_df['low_3m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(63).min())
mom_df['low_1m'] = mom_df['c'].groupby(level='insID').apply(lambda x: x/x.rolling(21).min())


mom_df = mom_df.groupby(level='insID').apply(
    lambda x: x.assign(acceleration = x.mom6 -  x.mom12))
mom_df = mom_df.groupby(level='insID').apply(
    lambda x: x.assign(adj_mom = x.mom12 -  x.mom1))
mom_df = mom_df.groupby(level='insID').apply(
    lambda x: x.assign(sharpe = x.mom12/x.volatility))


mom_df = mom_df.unstack(level ='d').fillna(mom_df.unstack(level ='d').median()).stack(level ='d')
mom_df = mom_df[['mom12', 'mom6', 'mom3', 'mom1','1_m_ret','volatility', 'acceleration',
               'adj_mom','sharpe', 'high_12m', 'high_6m', 'high_3m', 'high_1m', 'high_2y','high_3y',
                 'high_4y','low_1m', 'low_3m', 'low_6m', 'low_12m', 'low_2y']]

mom_df =mom_df.groupby([mom_df.index.get_level_values(0), pd.Grouper(freq='m', level=1)]).last()
mom_df = mom_df.swaplevel()
mom_df = mom_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

## Quality Metrics

In [30]:
margin_df = complete_df[['gross_Income','net_Sales']]

margin_df.loc[margin_df['net_Sales'] == 0,'net_Sales'] =-1
margin_df['Gross_margin'] = margin_df['gross_Income'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/margin_df['net_Sales'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())

margin_df = margin_df.unstack(level ='d').fillna(margin_df.unstack(level ='d').median()).stack(level ='d')
margin_df = margin_df[['Gross_margin']]
margin_df = margin_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
GPA_df = complete_df[['gross_Income','total_Assets']]
#GPA_df = GPA_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
GPA_df['GPA'] = GPA_df['gross_Income']/ GPA_df['total_Assets']

GPA_df['GPA'] = GPA_df['gross_Income'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/GPA_df['total_Assets'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())


GPA_df = GPA_df.unstack(level ='d').fillna(GPA_df.unstack(level ='d').median()).stack(level ='d')
GPA_df['GPA_chg'] = GPA_df['GPA'].diff(12)
GPA_df = GPA_df[['GPA', 'GPA_chg']]
GPA_df = GPA_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [32]:
ROE_df = complete_df[['earnings_Per_Share','total_Equity','number_Of_Shares']]
ROE_df['total_earnings'] = ROE_df['earnings_Per_Share'] * ROE_df['number_Of_Shares']
#ROE_df = ROE_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
ROE_df['ROE'] = ROE_df['total_earnings'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/ROE_df['total_Equity'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())




ROE_df = ROE_df.unstack(level ='d').fillna(ROE_df.unstack(level ='d').median()).stack(level ='d')
ROE_df['ROE_chg'] = ROE_df['ROE'].diff(12)
ROE_df = ROE_df[['ROE', 'ROE_chg']]
ROE_df = ROE_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [33]:
ROIC_df = complete_df[['operating_Income','total_Equity', 'net_Debt']]
#ROIC_df = ROIC_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
ROIC_df['EV'] =(ROIC_df['total_Equity'] + ROIC_df['net_Debt'] )
ROIC_df.loc[ROIC_df['EV'] == 0,'EV'] =-1
ROIC_df['ROIC'] = ROIC_df['operating_Income'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/ROIC_df['EV'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())




ROIC_df = ROIC_df.unstack(level ='d').fillna(ROIC_df.unstack(level ='d').median()).stack(level ='d')
ROIC_df = ROIC_df[['ROIC']]
ROIC_df = ROIC_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [34]:

AT_df = complete_df[['total_Assets','net_Sales']]
#AT_df = AT_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
AT_df['AT'] = AT_df['net_Sales'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/AT_df['total_Assets'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())



AT_df = AT_df.unstack(level ='d').fillna(AT_df.unstack(level ='d').median()).stack(level ='d')
AT_df = AT_df[['AT']]
AT_df = AT_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
#FCFROE

FCFROE_df = complete_df[['cash_Flow_For_The_Year','total_Equity']]
#FCFROE_df = FCFROE_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
FCFROE_df['FCFROE'] = FCFROE_df['cash_Flow_For_The_Year'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/FCFROE_df['total_Equity'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
FCFROE_df = FCFROE_df.unstack(level ='d').fillna(FCFROE_df.unstack(level ='d').median()).stack(level ='d')
FCFROE_df = FCFROE_df[['FCFROE']]
FCFROE_df = FCFROE_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [36]:
#ROA

ROA_df = complete_df[['earnings_Per_Share', 'number_Of_Shares', 'total_Assets']]
ROA_df['total_earnings'] = ROA_df['earnings_Per_Share'] * ROA_df['number_Of_Shares']
#ROA_df = ROA_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
ROA_df['ROA'] = ROA_df['total_earnings'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/ROA_df['total_Assets'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())

ROA_df = ROA_df.unstack(level ='d').fillna(ROA_df.unstack(level ='d').median()).stack(level ='d')
ROA_df = ROA_df[['ROA']]
ROA_df = ROA_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Valuation Metrics

In [37]:
PE_df = complete_df[['earnings_Per_Share','c']]
#PE_df = PE_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')

PE_df.loc[PE_df['earnings_Per_Share'] == 0,'earnings_Per_Share'] =-1
PE_df['PE'] = PE_df['c']/ PE_df['earnings_Per_Share'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())

PE_df = PE_df.unstack(level ='d').fillna(PE_df.unstack(level ='d').median()).stack(level ='d')
PE_df = PE_df[['PE']]
PE_df = PE_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [38]:
asset_g_df = complete_df[['total_Assets']]
#asset_g_df = asset_g_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
asset_g_df['Asset_growth'] = asset_g_df['total_Assets'].pct_change(12)
asset_g_df = asset_g_df.unstack(level ='d').fillna(asset_g_df.unstack(level ='d').median()).stack(level ='d')
asset_g_df = asset_g_df[['Asset_growth']]
asset_g_df = asset_g_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
PB_df = complete_df[['total_Equity','number_Of_Shares', 'c']]
#PB_df = PB_df.unstack(level ='insID').fillna(method ='ffill').stack(level ='insID')
PB_df['equity_ps'] = PB_df['total_Equity']/PB_df['number_Of_Shares']
PB_df.loc[PB_df['equity_ps'] == 0,'equity_ps'] =-1

PB_df['PB'] = PB_df['c']/ PB_df['equity_ps'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
PB_df = PB_df.unstack(level ='d').fillna(PB_df.unstack(level ='d').median()).stack(level ='d')
PB_df = PB_df[['PB']]
PB_df = PB_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
EVEBIT = complete_df[['operating_Income','total_Equity','number_Of_Shares', 'c', 'net_Debt']]
EVEBIT['EV'] = (EVEBIT['number_Of_Shares'] * EVEBIT['c']) + EVEBIT['net_Debt']

EVEBIT.loc[EVEBIT['operating_Income'] == 0,'operating_Income'] =-1

EVEBIT['EVEBIT'] = EVEBIT['EV'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/EVEBIT['operating_Income'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
EVEBIT = EVEBIT.unstack(level ='d').fillna(EVEBIT.unstack(level ='d').median()).stack(level ='d')
EVEBIT = EVEBIT['EVEBIT']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
EVFCF = complete_df[['cash_Flow_For_The_Year','total_Equity','number_Of_Shares', 'c', 'net_Debt']]

EVFCF['EV'] = (EVFCF['number_Of_Shares'] * EVFCF['c']) + EVFCF['net_Debt']
EVFCF.loc[EVFCF['cash_Flow_For_The_Year'] == 0,'cash_Flow_For_The_Year'] =-1

EVFCF['EVFCF'] = EVFCF['EV'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())/EVFCF['cash_Flow_For_The_Year'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())

EVFCF = EVFCF.unstack(level ='d').fillna(EVFCF.unstack(level ='d').median()).stack(level ='d')
EVFCF = EVFCF['EVFCF']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
#PS
PS_df = complete_df[['net_Sales','c', 'number_Of_Shares']]


PS_df.loc[PS_df['net_Sales'] == 0,'net_Sales'] =-1

PS_df['sales_per_share'] = PS_df['net_Sales']/PS_df['number_Of_Shares']
PS_df['PS'] = PS_df['c']/ PS_df['sales_per_share'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
PS_df = PS_df.unstack(level ='d').fillna(PS_df.unstack(level ='d').median()).stack(level ='d')
PS_df = PS_df[['PS']]
PS_df = PS_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
#PFCF

PFCF_df = complete_df[['cash_Flow_For_The_Year','c', 'number_Of_Shares']]

PFCF_df['fcf_per_share'] = PFCF_df['cash_Flow_For_The_Year']/PFCF_df['number_Of_Shares']

PFCF_df.loc[PFCF_df['cash_Flow_For_The_Year'] == 0,'fcf_per_share'] =-1
PFCF_df['PFCF'] = PFCF_df['c']/ PFCF_df['fcf_per_share'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
PFCF_df = PFCF_df.unstack(level ='d').fillna(PFCF_df.unstack(level ='d').median()).stack(level ='d')
PFCF_df = PFCF_df[['PFCF']]
PFCF_df = PFCF_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
#Div Yield
DIV_df = complete_df[['dividend','c', 'number_Of_Shares']]
DIV_df.loc[DIV_df['dividend'].isna(),['dividend']] = 0

DIV_df['rol_div'] = DIV_df['dividend'].groupby(level='insID').apply(lambda x: x.rolling(12).mean())
DIV_df['div_per_share'] = DIV_df['rol_div']/DIV_df['number_Of_Shares']
DIV_df['DIV_Y'] = DIV_df['div_per_share']/ DIV_df['c']
DIV_df = DIV_df.unstack(level ='d').fillna(DIV_df.unstack(level ='d').median()).stack(level ='d')
DIV_df = DIV_df[['DIV_Y']]
DIV_df = DIV_df.sort_index() 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

### Merge Data

In [45]:
export_df = mcap_df.copy()

export_list = [PE_df, GPA_df, asset_g_df,PB_df, EAR_date_df,EVEBIT,EVFCF,div_adj_df, ROIC_df, ROE_df, AT_df,
              list_sector_df, rev_year, eps_year, mom_df, FCFROE_df, ROA_df, PS_df, PFCF_df,
               DIV_df,volume_df, margin_df,norm_res_df,ido_mom_df, alpha_df, beta_df]

for df in export_list:
    try:
        export_df = export_df.join(df.swaplevel(), on=['d','insID'],how='left')
    except:
        export_df = export_df.join(df, on=['d','insID'],how='left')

In [46]:
SAVE = 'YES'

if SAVE == 'YES':
    export_df.to_csv("data/VQ_monthly_data.csv")

In [47]:
for col in export_df.columns:
    try:
        print(col + ' : ' + str(export_df[col].max()))
    except:
        print(col + ' is non-numerical')

c : 187500.0
number_Of_Shares : 8505.7
MCAP : 1617774.4000000001
PE : inf
GPA : 3.4592373727704095
GPA_chg : 6.765529409210369
Asset_growth : 6153.319517354854
PB : 36754.24103227928
EA_ret : 4.726950354609929
EAR_std : 53.91607704945541
EVEBIT : inf
EVFCF : inf
adj_c : 206075.125
ROIC : 12015.285714289323
ROE : 4665.9653056
ROE_chg : 4666.064003006776
AT : 4.818116669055468
Name is non-numerical
sector is non-numerical
market is non-numerical
RSURP_Y : 971.8952692841466
rev_growth : 578.4940828402366
SUE_Y : 4168.465798624778
EPS_growth : 840.675
EPS_acceleration : 499.30889884763127
mom12 : 80.74640770566873
mom6 : 56.81226228510574
mom3 : 24.17876160048684
mom1 : 17.68
1_m_ret : 1.7689741451209344
volatility : 27.362023448221567
acceleration : 37.61721611721612
adj_mom : 80.62906877233137
sharpe : inf
high_12m : 1.0
high_6m : 1.0
high_3m : 1.0
high_1m : 1.0
high_2y : 1.0
high_3y : 1.0
high_4y : 1.0
low_1m : 22.238095238095237
low_3m : 25.201766407796562
low_6m : 57.2893110206543
low

In [48]:
for col in export_df.columns:
    try:
        print(col + ' : ' + str(export_df[col].min()))
    except:
        print(col + ' is non-numerical')

c : 0.01
number_Of_Shares : 0.0021
MCAP : 0.01334
PE : -1.4843864371813156e+18
GPA : -6.5017006802721085
GPA_chg : -6.712037046173784
Asset_growth : -0.9999908653997672
PB : -11486.325605207745
EA_ret : -0.6978661493695442
EAR_std : -10.156587299878137
EVEBIT : -inf
EVFCF : -1.6811430300137583e+19
adj_c : 0.01
ROIC : -5973.826833057431
ROE : -715.2758222973001
ROE_chg : -4663.978473469298
AT : -0.005954783522100052
Name is non-numerical
sector is non-numerical
market is non-numerical
RSURP_Y : -4990.192426956073
rev_growth : -3223405.0
SUE_Y : -2751.199609479129
EPS_growth : -1795.8
EPS_acceleration : -1796.1447573635326
mom12 : -0.9824519230769231
mom6 : -0.9693932230102443
mom3 : -0.953671875
mom1 : -0.9543846153846154
1_m_ret : -0.7463768115942029
volatility : 0.0
acceleration : -78.71301519276348
adj_mom : -14.146019417475728
sharpe : -inf
high_12m : 0.014777327935222672
high_6m : 0.030486656200941913
high_3m : 0.04360294117647059
high_1m : 0.04360294117647059
high_2y : 0.004310344